# Solidity Ast

In [27]:
import json

In [2]:
def read_file(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    return data

In [23]:
d = read_file("test/c.forge.ast.json")
sources = d['sources']
id_to_path = d['build_infos'][0]['source_id_to_path']

In [25]:
def cache_ids(sources):
    nodes = {}
    for p, contents in sources.items():
        ast = sources[p][0]['source_file']['ast']
        if "id" in ast:
            nodes[ast["id"]] = {'src': ast['src'] }
            # print(ast['id'])
        curr = [ast]
        while curr:
            tree = curr.pop()
            # process current node
            nodes[tree['id']] = {'src': tree['src'] }
        
            if 'nameLocation' in tree:
                nodes[tree['id']]['nameLocation'] = tree['nameLocation']
            if 'referencedDeclaration' in tree:
                nodes[tree['id']]['referencedDeclaration'] = tree['referencedDeclaration']
            if 'nodeType' in tree:
                nodes[tree['id']]['nodeType'] = tree['nodeType']
            if 'memberLocation' in tree:
                nodes[tree['id']]['memberLocation'] = tree['memberLocation']

            # check for nodes
            if "nodes" in tree:
                for node in tree['nodes']:
                    # print(node['id'])
                    curr.append(node)
            # check for struct members nodes
            if "members" in tree:
                for node in tree['members']:
                    # print(node['id'])
                    curr.append(node)
            # check for declaration nodes
            if "declarations" in tree:
                for node in tree['declarations']:
                    # print(node['id'])
                    curr.append(node)
            if "symbolAliases" in tree:
                for node in tree["symbolAliases"]:
                    if "foreign" in node:
                        curr.append(node["foreign"])
            # check library using for directive
            if "libraryName" in tree:
                curr.append(tree["libraryName"])
            # check for body nodes
            if "body" in tree:
                for node in tree['body']['nodes']:
                    # print(node['id'])
                    curr.append(node)
                for node in tree['body']['statements']:
                    # print(node['id'])
                    curr.append(node)
            # check for body nodes (incomplete)
            if "expression" in tree:
                curr.append(tree["expression"])
                if "arguments" in tree["expression"]:
                    for node in tree['expression']['arguments']:
                        # print(node['id'])
                        curr.append(node)
            # check left
            if "leftHandSide" in tree:
                curr.append(tree["leftHandSide"])
            # check right
            if "rightHandSide" in tree:
                curr.append(tree["rightHandSide"])
            # check for body statement nodes
            if "statements" in tree:
                for node in tree['statements']:
                    # print(node['id'])
                    curr.append(node)
            # check for parameter nodes
            if "parameters" in tree:
                for node in tree['parameters']['parameters']:
                    # print(node['id'])
                    curr.append(node)
            # # check for return parameter nodes
            if "returnParameters" in tree:
                if "returnParameters" in tree['returnParameters']:
                    for node in tree['returnParameters']['returnParameters']:
                        # print(node['id'])
                        curr.append(node)
    return nodes
    
def goto(path, position):
    # (line, char) = position
    p = path.split("file://")[1]
    nodes = cache_ids(sources)

    refs = {}
    for i, content in nodes.items():
        if 'referencedDeclaration' not in content:
            continue
        start_b, l, s = content['src'].split(":")
        end_b = int(start_b) + int(l)
        
        # print(i, content)
        if int(start_b) <= int(position) < int(end_b):
            diff = int(end_b) - int(start_b)
            if i not in refs:
                refs[diff] = i
            else:
                if refs[diff] <= i:
                    refs[diff] = i

    # print(refs)
    location, file = None, None
    if refs:
        choice = nodes[refs[min(refs)]]
        ref = choice['referencedDeclaration']
        node = nodes[ref]
        if "nameLocation" in node:
            location, _, file = node['nameLocation'].split(":")
        elif "src" in node:
            location,_, file = node['src'].split(":")
        else:
            # default to same location
            location = int(position)
        return id_to_path[file], int(location)
    else:
        return path.split("/")[-1], int(position)

In [26]:
tests = (
    # name.add_one(votes);
    (("file:///Users/meek/Developer/lsp/C.sol", 430),("C.sol", 371)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 431),("C.sol", 371)), 
    (("file:///Users/meek/Developer/lsp/C.sol", 432),("C.sol", 371)),
    (("file:///Users/meek/Developer/lsp/C.sol", 433),("C.sol", 371)),
    (("file:///Users/meek/Developer/lsp/C.sol", 434),("B.sol", 247)), # .add_one
    (("file:///Users/meek/Developer/lsp/C.sol", 435),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 436),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 437),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 438),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 439),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 440),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 441),("B.sol", 247)),
    (("file:///Users/meek/Developer/lsp/C.sol", 442),("C.sol", 442)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 443),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 444),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 445),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 446),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 447),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 448),("C.sol", 448)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 449),("C.sol", 449)), # not a node
    # name.get_votes(votes);
    (("file:///Users/meek/Developer/lsp/C.sol", 466),("C.sol", 371)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 467),("C.sol", 371)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 468),("C.sol", 371)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 469),("C.sol", 371)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 470),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 471),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 472),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 473),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 474),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 475),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 476),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 477),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 478),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 479),("B.sol", 359)), # get_votes
    (("file:///Users/meek/Developer/lsp/C.sol", 480),("C.sol", 480)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 481),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 482),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 483),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 484),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 485),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 486),("C.sol", 486)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 487),("C.sol", 487)), # not a node
    # bool fad;
    (("file:///Users/meek/Developer/lsp/C.sol", 417),("C.sol", 417)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 418),("C.sol", 418)), # not a node
    (("file:///Users/meek/Developer/lsp/C.sol", 419),("C.sol", 419)), # not a node
    # name("meek");
    (("file:///Users/meek/Developer/lsp/C.sol", 314),("A.sol", 260)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 315),("A.sol", 260)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 316),("A.sol", 260)), # name
    (("file:///Users/meek/Developer/lsp/C.sol", 317),("A.sol", 260)), # name
    # votes.name = "2024 Elections";
    (("file:///Users/meek/Developer/lsp/C.sol", 275),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 276),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 277),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 278),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 279),("C.sol", 212)), # votes
    (("file:///Users/meek/Developer/lsp/C.sol", 280),("B.sol", 163)), # .name
    (("file:///Users/meek/Developer/lsp/C.sol", 281),("B.sol", 163)), # .name
    (("file:///Users/meek/Developer/lsp/C.sol", 282),("B.sol", 163)), # .name
    (("file:///Users/meek/Developer/lsp/C.sol", 283),("B.sol", 163)), # .name
    (("file:///Users/meek/Developer/lsp/C.sol", 284),("B.sol", 163)), # .name
    # using D for *;
    (("file:///Users/meek/Developer/lsp/C.sol", 146),("B.sol", 66)), # D
    # (("file:///Users/meek/Developer/lsp/C.sol", (21,8)),("file:///Users/meek/Developer/lsp/C.sol", (19,36), 371)),
    # (("file:///Users/meek/Developer/lsp/C.sol", (21,9)),("file:///Users/meek/Developer/lsp/C.sol", (19,36), 371)),
    # (("file:///Users/meek/Developer/lsp/C.sol", (21,10)),("file:///Users/meek/Developer/lsp/C.sol", (19,36), 371)),
    # (("file:///Users/meek/Developer/lsp/C.sol", (21,11)),("file:///Users/meek/Developer/lsp/C.sol", (19,36), 371)),
)

In [21]:
for test in tests:
    res = goto(test[0][0], test[0][1])

    if res != test[1]:
        print(f"❌ Fail Expected {test[1][1]}, got {res[1]}, Position {test[0][1]} {res[0]}")
    else:
        print(f"✅ Success {test[1][1]} == {res[1]}, Position {test[0][1]} {res[0]}")
    assert res == test[1], "failed"

✅ Success 371 == 371, Position 430 C.sol
✅ Success 371 == 371, Position 431 C.sol
✅ Success 371 == 371, Position 432 C.sol
✅ Success 371 == 371, Position 433 C.sol
✅ Success 247 == 247, Position 434 B.sol
✅ Success 247 == 247, Position 435 B.sol
✅ Success 247 == 247, Position 436 B.sol
✅ Success 247 == 247, Position 437 B.sol
✅ Success 247 == 247, Position 438 B.sol
✅ Success 247 == 247, Position 439 B.sol
✅ Success 247 == 247, Position 440 B.sol
✅ Success 247 == 247, Position 441 B.sol
✅ Success 442 == 442, Position 442 C.sol
✅ Success 212 == 212, Position 443 C.sol
✅ Success 212 == 212, Position 444 C.sol
✅ Success 212 == 212, Position 445 C.sol
✅ Success 212 == 212, Position 446 C.sol
✅ Success 212 == 212, Position 447 C.sol
✅ Success 448 == 448, Position 448 C.sol
✅ Success 449 == 449, Position 449 C.sol
✅ Success 371 == 371, Position 466 C.sol
✅ Success 371 == 371, Position 467 C.sol
✅ Success 371 == 371, Position 468 C.sol
✅ Success 371 == 371, Position 469 C.sol
✅ Success 359 ==